In [1]:
import cobra
from cobra import Metabolite, Reaction
import gurobipy

def make_reaction (df, model):
    """
    df: Takes in a dataframe with two columns; metabolite IDs in one and their coefficients (mmol/gDW) in the other. 
    model: Takes in the model that contains the metabolite objects
    Return: This function will returen a reaction with metabolite objects (from the metabolite IDs in the dataframes) with corresponding coefficients from df.
    Essentially pairs the coefficients to the metabolite objects already in the model. Important so that both coefficients and other potential attributes already
    in the model are all paired to the metabolite."""

    metabolite_to_coef = {} #Dictionary, maps metabolite object to coef. Float.
    for list in df.values.tolist():
        metabolite_to_coef[model.metabolites.get_by_id(list[1])] = float(list[0])


    reaction = Reaction ()
    reaction.add_metabolites(metabolite_to_coef)
    return reaction

def modify_reaction (reaction, reaction_id, model):

    """
    Reaction: The reaction object
    Reaction_id: The ID of the reaction where the attributes are gathered from.
    Model: The model that contains the reaction object with reaction ID.
    Return: Returns object with the same id, name, substystem, lower- and upper bound as reaction object from model with id = reaction_id.
    Returns a rection variable that has the same attributes as the corresponding reaction from the model. Creates a new reaction object that 
    is connected to the attributes of the existing reaction in the model.
    """

    reaction.id = model.reactions.get_by_id(reaction_id).id + "new"
    reaction.name = model.reactions.get_by_id(reaction_id).name
    reaction.subsystem = model.reactions.get_by_id(reaction_id).subsystem
    reaction.lower_bound = model.reactions.get_by_id(reaction_id).lower_bound
    reaction.upper_bound = model.reactions.get_by_id(reaction_id).upper_bound
    return reaction


def new_synthesis_reaction(df, rxn_id, model):
    """
    :param df: DataFrame with two columns: Coefficient (mmol/gDW) and metabolite ID.
    :param rxn_id: Reaction ID of the original synthesis Reaction object.
    :param model: The metabolic model in use.
    :return: New synthesis Reaction object with the metabolites and coefficients from df.
    """
    synthesis_reaction = make_reaction(df, model)
    synthesis_reaction = modify_reaction(synthesis_reaction, rxn_id, model)

    return synthesis_reaction

In [2]:
import cobra
from cobra import Reaction, Metabolite
import pandas as pd 
import os

os.getcwd()
os.chdir("C:/Users/sofie/OneDrive/Documents/Master - Data/")

#I take in two models, so that I have one that will retain the original BOF from the model to compare results with. The iBsu1147mann will have the updated, experimentally derived coefficients.
#The iBsu1147ogm model will keep the original BOF.
iBsu1147mann = cobra.io.read_sbml_model("iBsu1147.xml")    #model with new BOF
iBsu1147ogm = cobra.io.read_sbml_model("iBsu1147.xml")     #model with original BOF

#The names and IDs are mixed up. In the code below this is corrected for. The metabolite.name function will provde the name of the metabolite. 
# The metabolite.id will now return the ID of the metabolite.
for metabolite in iBsu1147mann.metabolites:
    name = metabolite.id
    id = metabolite.name

    metabolite.name = name
    metabolite.id = id

for metabolite in iBsu1147ogm.metabolites:
    name = metabolite.id
    id = metabolite.name

    metabolite.name = name
    metabolite.id = id

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-25


In [3]:
#COFACTORS AND IONS:
#I use the functions that have been defined earlier to create the cofactor reaction. This reaction is not a part of the original BOF. It does not contain any new/updated numbers. It has just had all the substrates 
#from the origional BOF that are metabolites, cofactors or ions lumped together to one synthesis reaction to ease the process of modifying the BOF.

os.chdir("C:/Users/sofie/OneDrive/Documents/Master - Data/")

cofactor = Metabolite( id="Cofactor[c]", formula= None, name="cofactor", charge= None, compartment= "c")
iBsu1147mann.add_metabolites(cofactor)

#Because the cofactor metabolite and reaction are not a part of the model, a new metabolite had to be made and added to the iBsu1147gly.
cofactors_ions_df = pd.read_excel("Macromolecule-synthesis.xlsx","Cofactor") 
cofac_synthesis= make_reaction(cofactors_ions_df, iBsu1147mann)
cofac_synthesis.name = "cofac_synthesis"
cofac_synthesis.id = "cofactor"
cofac_synthesis.lower_bound = -1000.0
cofac_synthesis.upper_bound = 1000

print(cofac_synthesis)

cofactor: 0.406841373641698 C00003[c] + 0.005285365155927 C00005[c] + 0.0235128411879642 C00006[c] + 0.00284963130743174 C00010[c] + 0.117296253333691 C00020[c] + 0.00494595852149485 C00035[c] + 0.0109034381311329 C00044[c] + 0.0257457795723863 C00055[c] + 0.0136320888368966 C00063[c] + 0.0666085520073099 C00076[c] + 0.00652911183605009 C00112[c] + 0.0132614210650826 C00144[c] + 0.00461840878988398 C00234[c] + 14.6838028159594 C00238[c] + 2.11548582540145 C00305[c] + 0.00334427181834891 C00828[c] + 0.0716549927561037 C14819[c] <=> Cofactor[c]


In [4]:
#DNA:
#Constructing the new DNA synthesis reaction using the previously defined functions.
DNA_df = pd.read_excel("Macromolecule-synthesis.xlsx","DNA")
DNA_synthesis = make_reaction(DNA_df, iBsu1147mann)
modify_reaction(DNA_synthesis, "rxn05294", iBsu1147mann)
print(DNA_synthesis)

rxn05294new: 0.925419454369164 C00131[c] + 0.700555089212493 C00286[c] + 0.699717605543381 C00458[c] + 0.921964834234075 C00459[c] <=> 3.24765698335911 C00013[c] + C00039[c]


In [5]:
#RNA:
#Constructing the new RNA synthesis reaction using the previously defined functions.
RNA_df = pd.read_excel("Macromolecule-synthesis.xlsx","RNA")
RNA_synthesis = make_reaction(RNA_df, iBsu1147mann)

RNA_synthesis = modify_reaction(RNA_synthesis, "rxn05295", iBsu1147mann)

print(RNA_synthesis)

rxn05295new: 0.810318111715803 C00002[c] + 0.998544094063492 C00044[c] + 0.615467416022917 C00063[c] + 0.665731114102355 C00075[c] --> 3.09006073590457 C00013[c] + cpd11462[c]


In [6]:
#Lipid:
Lipid_df = pd.read_excel("Macromolecule-synthesis.xlsx","Lipid")
Lipid_synthesis = make_reaction(Lipid_df, iBsu1147mann)

Lipid_synthesis = modify_reaction(Lipid_synthesis, "rxn10201", iBsu1147mann)
print(Lipid_synthesis)
print(iBsu1147mann.reactions.get_by_id("rxn10201"))

rxn10201new: 0.0182341974255657 cpd15529[c] + 0.0735693637318944 cpd15531[c] + 0.0249314494535681 cpd15533[c] + 0.00572228475376097 cpd15536[c] + 0.0231742993712587 cpd15538[c] + 0.00787805688331436 cpd15540[c] + 0.0539093645252547 cpd15695[c] + 0.126715623078546 cpd15696[c] + 0.00871948075130027 cpd15697[c] + 0.146496112576544 cpd15698[c] + 0.248310408774362 cpd15699[c] + 0.0342393244610017 cpd15700[c] + 0.00874458289533326 cpd15707[c] + 0.00210355966996477 cpd15708[c] + 0.00304137577103737 cpd15709[c] + 0.00649442670421581 cpd15710[c] + 0.0152621035720594 cpd15711[c] + 0.00100609393284234 cpd15712[c] + 0.0171698665185669 cpd15713[c] + 0.0290883645054317 cpd15714[c] + 0.00406955959062874 cpd15715[c] + 0.0170092127967557 cpd15722[c] + 0.0399826950157504 cpd15723[c] + 0.0027371377853575 cpd15724[c] + 0.046077495586961 cpd15725[c] + 0.0780777088002198 cpd15726[c] + 0.0107838810765736 cpd15727[c] + 0.014529120966296 cpd15728[c] + 0.00353136962256138 cpd15729[c] + 0.00500938386322398 cpd15

In [7]:
#Lipoteichoic acid:

#print(iBsu1147.reactions.get_by_id("rxn10200"))
Lipo_df = pd.read_excel("Macromolecule-synthesis.xlsx", "Lipoteichoic_acid")
Lipo_synthesis = make_reaction(Lipo_df, iBsu1147mann)
Lipo_synthesis = modify_reaction(Lipo_synthesis, "rxn10200", iBsu1147mann)

print(Lipo_synthesis)
print(iBsu1147mann.reactions.get_by_id("rxn10200"))

rxn10200new: 0.00488084303736876 cpd15746[c] + 0.00112542250060168 cpd15747[c] + 0.00176637167875183 cpd15748[c] + 0.00369824666641567 cpd15749[c] + 0.00869343744448727 cpd15750[c] + 0.000538136516553297 cpd15751[c] + 0.00938453914048955 cpd15752[c] + 0.0158983481591235 cpd15753[c] + 0.0022709060083442 cpd15754[c] + 0.00227592126012941 cpd15755[c] + 0.0005216864906978 cpd15756[c] + 0.000828218679809985 cpd15757[c] + 0.00172925881554125 cpd15758[c] + 0.00406536309709322 cpd15759[c] + 0.000249558928832172 cpd15760[c] + 0.00436326905313484 cpd15761[c] + 0.00739248113140316 cpd15762[c] + 0.00105922117703687 cpd15763[c] + 0.00203819832551034 cpd15764[c] + 0.00046691994120328 cpd15765[c] + 0.000742357569247148 cpd15766[c] + 0.00154971280163065 cpd15767[c] + 0.00364207584642129 cpd15768[c] + 0.000223379314513363 cpd15769[c] + 0.0039068811406805 cpd15770[c] + 0.0066191293061234 cpd15771[c] + 0.000948584722655084 cpd15772[c] + 0.00646064734971069 cpd15773[c] + 0.00148451452842289 cpd15774[c] + 

In [8]:
#Cell wall:

Cellwall_df = pd.read_excel("Macromolecule-synthesis.xlsx","Cell_wall")
Cellwall_synthesis = make_reaction(Cellwall_df, iBsu1147mann)
Cellwall = modify_reaction(Cellwall_synthesis, "rxn10198", iBsu1147mann)

print(Cellwall)

rxn10198new: 0.0145386839210681 cpd11459[c] + 0.454208539051299 cpd15665[c] + 0.0160426857060061 cpd15667[c] + 0.0112298799942043 cpd15668[c] + 0.0081015562815331 cpd15669[c] <=> cpd15664[c] + 0.489582661033042 cpd15666[c]


In [9]:
#proteins Glycerol:
print(iBsu1147mann.reactions.get_by_id("rxn05296"))
PMann_df = pd.read_excel("macromolecule-synthesis.xlsx", "Proteins_Mann")
PMann_synthesis = make_reaction(PMann_df, iBsu1147mann)

PMann_synthesis = modify_reaction(PMann_synthesis, "rxn05296", iBsu1147mann)
print(PMann_synthesis)


rxn05296: 0.5051 C00886[c] + 0.6114 C01931[c] + 0.6555 C02047[c] + 0.3653 C02163[c] + 0.4928 C02282[c] + 0.7723 C02412[c] + 0.2145 C02430[c] + 0.4091 C02553[c] + 0.5807 C02554[c] + 0.3041 C02702[c] + 0.2097 C02839[c] + 0.2801 C02984[c] + 0.4928 C02987[c] + 0.1546 C02988[c] + 0.3526 C02992[c] + 0.1073 C03125[c] + 0.5107 C03127[c] + 0.2801 C03402[c] + 0.3329 C03511[c] + 0.1028 C03512[c] <=> C00017[c] + 0.2097 C00787[c] + 0.5051 C01635[c] + 0.3653 C01636[c] + 0.2801 C01637[c] + 0.2801 C01638[c] + 0.1073 C01639[c] + 0.4928 C01640[c] + 0.4928 C01641[c] + 0.7723 C01642[c] + 0.1546 C01643[c] + 0.5107 C01644[c] + 0.6555 C01645[c] + 0.6114 C01646[c] + 0.2145 C01647[c] + 0.3329 C01648[c] + 0.3041 C01649[c] + 0.4091 C01650[c] + 0.3526 C01651[c] + 0.1028 C01652[c] + 0.5807 C01653[c]
rxn05296new: 0.924181224557816 C00886[c] + 0.529528614020619 C01931[c] + 0.586236764447444 C02047[c] + 0.344630811549746 C02163[c] + 0.396831479423161 C02282[c] + 0.582012983839015 C02412[c] + 0.216674871149239 C02430[

In [10]:
#Biomass objective function:

Datafile = pd.read_excel("BOF_ny.xlsx", "Mannitol")

BOF_df = pd.DataFrame(Datafile, columns= ["g/gDW", "BOF-ID"])
                      
BOFrx = make_reaction(BOF_df, iBsu1147mann)

BOFmann = modify_reaction(BOFrx, "bio00006", iBsu1147mann)
print(BOFmann)

bio00006new: 105.0 C00001[c] + 105.003 C00002[c] + 0.518547974140424 C00017[c] + 0.0484219374456551 C00039[c] + 0.000273109 C00229[c] + 0.0446336317745074 Cofactor[c] + 0.0899574750752081 cpd11462[c] + 0.2242 cpd15664[c] + 0.0304 cpd15670[c] + 0.0336668435453882 cpd15800[c] --> 104.997 C00008[c] + 104.987 C00009[c] + 0.000273109 C03688[c]


In [11]:
#Removing old BOF-synthesis reactions
SynthesisIDs = ["rxn05294", "rxn05295", "rxn05296", "rxn10198", "rxn10200", "rxn10201", "bio00006"]
for IDs in SynthesisIDs:
    iBsu1147mann.remove_reactions([IDs])

print(len(iBsu1147mann.reactions))

1736


c:\Users\sofie\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\core\group.py:107: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [12]:
#Adding new reactions:  
synthesis_reactions = (cofac_synthesis, DNA_synthesis, RNA_synthesis, Lipid_synthesis, Lipo_synthesis, Cellwall, PMann_synthesis, BOFmann)

for reactions in synthesis_reactions:
    iBsu1147mann.add_reactions([reactions])

print(len(iBsu1147mann.reactions))

1744


In [13]:
# importing df that contains the components of the minimal medium used experimentally
os.getcwd()
os.chdir("C:/Users/sofie/OneDrive/Documents/Master - Data/")

experimental_medium_df = pd.read_excel("Medium.xlsx", "Mannitol")

Exchange_reactions_medium = experimental_medium_df["Exchange reaction ID"]

exchange_reactions = Exchange_reactions_medium.values.tolist()

#The bacteria in the batch fermentation experiments were incubated in minimal media with their respecitive C-source.
#In order to make sure the model predictions are based on what was available to the bacteria during the experiments,
#the flux ranges for the exchange reactions for components in the media must be set to -1000 and 1000. 
#That means that the components that were in the cultivation media, is the same as what is "available" in the model.

for exchange in iBsu1147mann.exchanges:
    if exchange.id in exchange_reactions:  # if the component from the exchange reaction was
        # available in the medium, the model should be able to take up and secrete it.
        iBsu1147mann.reactions.get_by_id(exchange.id).lower_bound = -1000.0
        iBsu1147mann.reactions.get_by_id(exchange.id).upper_bound = 1000.0
    else:
        # if the exchange reaction is not part of the medium, it can't be taken up from the media. However,
        # it is possible for the bacteria to secrete it.
        iBsu1147mann.reactions.get_by_id(exchange.id).lower_bound = 0.0
        iBsu1147mann.reactions.get_by_id(exchange.id).upper_bound = 1000.0

#changing the medium composition for the model with the original BOF:
for exchange in iBsu1147ogm.exchanges:
    if exchange.id in exchange_reactions: 
        iBsu1147ogm.reactions.get_by_id(exchange.id).lower_bound = -1000.0
        iBsu1147ogm.reactions.get_by_id(exchange.id).upper_bound = 1000.0
    else:
        iBsu1147ogm.reactions.get_by_id(exchange.id).lower_bound = 0.0
        iBsu1147ogm.reactions.get_by_id(exchange.id).upper_bound = 1000.0

"""
for exchange in iBsu1147.exchanges:
    print(exchange.id, exchange.lower_bound, exchange.upper_bound)"""

'\nfor exchange in iBsu1147.exchanges:\n    print(exchange.id, exchange.lower_bound, exchange.upper_bound)'

In [14]:
#Setting the new objective function:
iBsu1147mann.reactions.get_by_id("bio00006new").lower_bound = -1000.0
iBsu1147mann.reactions.get_by_id("bio00006new").upper_bound = 1000.0

iBsu1147mann.objective = "bio00006new"
print(iBsu1147mann.objective)
print(iBsu1147mann.optimize())

New_reactions = [cofac_synthesis, DNA_synthesis, RNA_synthesis, Lipid_synthesis, Lipo_synthesis, Cellwall, PMann_synthesis, BOFmann]

for reaction in New_reactions:
    print(reaction.id, iBsu1147mann.optimize().fluxes[reaction.id])

Maximize
1.0*bio00006new - 1.0*bio00006new_reverse_1501f
<Solution 22.531 at 0x1da236b9520>
cofactor 1.0056292989792341
rxn05294new 1.090982675456458
rxn05295new 2.02680958284672
rxn10201new 0.7585393105458522
rxn10200new 0.684934868025449
rxn10198new 5.051394651687687
rxn05296new 11.683275928708397
bio00006new 22.530752237679245


In [15]:
print(iBsu1147mann.objective)
iBsu1147mann.optimize()

SynthesisIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

for reactionID in SynthesisIDs:
    print("Flux through reaction", reactionID, ":", iBsu1147mann.reactions.get_by_id(reactionID).flux)

Maximize
1.0*bio00006new - 1.0*bio00006new_reverse_1501f
Flux through reaction rxn05294new : 1.090982675456458
Flux through reaction rxn05295new : 2.02680958284672
Flux through reaction rxn05296new : 11.683275928708397
Flux through reaction rxn10198new : 5.051394651687687
Flux through reaction rxn10200new : 0.684934868025449
Flux through reaction rxn10201new : 0.7585393105458522
Flux through reaction bio00006new : 22.530752237679245
Flux through reaction cofactor : 1.0056292989792341


In [16]:
print(iBsu1147ogm.objective)
iBsu1147ogm.optimize()

SynthesisIDsog = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006"]

for reactionID in SynthesisIDsog:
    print("Flux through reaction", reactionID, ":", iBsu1147ogm.reactions.get_by_id(reactionID).flux)

Maximize
1.0*bio00006 - 1.0*bio00006_reverse_45381
Flux through reaction rxn05294 : 0.0026
Flux through reaction rxn05295 : 0.00655
Flux through reaction rxn05296 : 0.05284
Flux through reaction rxn10198 : 0.022420000000000002
Flux through reaction rxn10200 : 0.00304
Flux through reaction rxn10201 : 0.0076
Flux through reaction bio00006 : 0.1


In [ ]:
#Vekstrate:
iBsu1147mann.reactions.get_by_id("bio00006new").upper_bound = 0.443
iBsu1147mann.reactions.get_by_id("bio00006new").lower_bound = 0.379  #Model will not run with the measured growth rate. Setting the lower bound to 0 to see what the model suggests.

iBsu1147mann.optimize()

SynthesisIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

for reactionID in SynthesisIDs:
    print("Flux through reaction", reactionID, ":", iBsu1147mann.reactions.get_by_id(reactionID).flux)

In [ ]:
#Growth rate for model with original BOF:
iBsu1147ogm.reactions.get_by_id("bio00006").upper_bound = 0.443
iBsu1147ogm.reactions.get_by_id("bio00006").lower_bound = 0.379
iBsu1147ogm.optimize()


SynthesisIDsogm = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006"]

for reactionID in SynthesisIDsogm:
    print("Flux through reaction", reactionID, ":", iBsu1147ogm.reactions.get_by_id(reactionID).flux)

In [ ]:
#Setting the experimentally measured fluxes
import cobra

#This function takes in the model and a dictionary of measured fluxes and updates the fluxes for the reactions. 
def update_fluxes_in_model(model, fluxes):
    for reaction_id, flux in fluxes.items():
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            lower_bound, upper_bound = flux  # Extract lower and upper bounds from the tuple
            reaction.lower_bound = lower_bound
            reaction.upper_bound = upper_bound
            #print(f"Flux updated for reaction ID {reaction_id}.")
        else:
            print(f"Reaction ID {reaction_id} not found in the model.")

"""
Fluxes = {"E00096": (0, 1000), "E00068": (0, 1000), "E00117": (-4.78, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.01),
          "E00032": (-0.013, 1000), "E00009": (-1.842, 1000), "E00002": (-5.6, 1000), "E00004": (0, 5.0), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147mann, Fluxes)  # Update fluxes in the model using the Fluxes dictionary

iBsu1147mann.optimize()"""


In [18]:
#Setting the experimentally measured fluxes for original model

#This function takes in the model and a dictionary of measured fluxes and updates the fluxes for the reactions. 
def update_fluxes_in_model(model, fluxes):
    for reaction_id, flux in fluxes.items():
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            lower_bound, upper_bound = flux  # Extract lower and upper bounds from the tuple
            reaction.lower_bound = lower_bound
            reaction.upper_bound = upper_bound
            #print(f"Flux updated for reaction ID {reaction_id}.")
        else:
            print(f"Reaction ID {reaction_id} not found in the model.")

"""
Fluxes =  {"E00096": (0, 1000), "E00068": (0, 1000), "E00117": (-4.78, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.01),
          "E00032": (-0.013, 1000), "E00009": (-1.842, 1000), "E00002": (-5.6, 1000), "E00004": (0, 5.0), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147ogm, Fluxes)  # Update fluxes in the model using the Fluxes dictionary

iBsu1147ogm.optimize()"""

'\nFluxes =  {"E00096": (0, 1000), "E00068": (0, 1000), "E00117": (-4.78, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.01),\n          "E00032": (-0.013, 1000), "E00009": (-1.842, 1000), "E00002": (-5.6, 1000), "E00004": (0, 5.0), "EX_Methanol[c]": (0, 1000)}\n\nupdate_fluxes_in_model(iBsu1147ogm, Fluxes)  # Update fluxes in the model using the Fluxes dictionary\n\niBsu1147ogm.optimize()'

In [ ]:
import reframed 
from reframed import from_cobrapy
from reframed import FBA
from reframed import MOMA

cbmodelmann = reframed.from_cobrapy(iBsu1147mann)

#MOMA to get closest feasible solution because the model cannot run with the measured fluxes. 
Fluxes = {"E00117": -4.787, "E00096":0.0, "E00068":0.0, "E00017": 0.0, "E00046":0.0, "E00012":  3.01, "E00032": -0.013, "E00009": -1.842, "E00002": -5.6, "E00004": 5.0, "EX_Methanol[c]": 0.0, "bio00006new": 0.411}

ReactionIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor", "E00117", "E00012", "E00032", "E00009", "E00002", "E00004"]

modelMOMA = MOMA(cbmodelmann, reference= Fluxes, reactions=ReactionIDs)
print(modelMOMA)
for reactions in ReactionIDs:
    print(modelMOMA.show_values(reactions))

In [ ]:
#The new suggested fluxes from the MOMA analysis are:

#Growth rate = 0.224
iBsu1147mann.reactions.get_by_id("bio00006new").upper_bound = 0.379
iBsu1147mann.reactions.get_by_id("bio00006new").lower_bound = 0.37   #infeasible if i lock both upper and lower bound at 0.379

new_Fluxes = {"E00096": (0, 1000), "E00068": (0, 1000), "E00117": (-6.798, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.01),
          "E00032": (-0.0158, 1000), "E00009": (-2.093, 1000), "E00002": (-9.119, 1000), "E00004": (0, 6.005), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147mann, new_Fluxes)

print(iBsu1147mann.optimize())

cobra.io.write_sbml_model(iBsu1147mann, "C:/Users/sofie/OneDrive/Documents/Master - Data/ModelMan.sbml")

"""
#Running a FBA analysis with the MOMA-fluxes: 
cbmodelmann2 = reframed.from_cobrapy(iBsu1147mann)
solmann2 = FBA(cbmodelmann2)
#print(solmann2.show_values())

SynthesisIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor",  "E00096", "E00068", "E00017", "E00117", "E00046", "E00012", "E00008", "E00032", "E00009", "E00002", "E00004"]

for reactionID in SynthesisIDs:
    flux = solmann2.values.get(reactionID)
    print(reactionID, round(flux,6))"""

In [ ]:
#Performing a MOMA analysis for the model with the original BOF as well to get a feasible solution for further analysis.
#MOMA to get closest feasible solution because the model cannot run with the measured fluxes. 

import reframed 
from reframed import from_cobrapy
from reframed import FBA
from reframed import MOMA


cbmodelogmann = reframed.from_cobrapy(iBsu1147ogm)
Fluxes = {"E00117": -4.787, "E00096":0.0, "E00068":0.0, "E00017": 0.0, "E00046":0.0, "E00012":  3.01, "E00032": -0.013, "E00009": -1.842, "E00002": -5.6, "E00004": 5.0, "EX_Methanol[c]": 0.0}

ReactionIDs = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006", "E00117", "E00012", "E00032", "E00009", "E00002", "E00004"]

modelMOMAog = MOMA(cbmodelogmann, reference= Fluxes, reactions=ReactionIDs)
print(modelMOMAog)
for reactions in ReactionIDs:
    print(modelMOMAog.show_values(reactions))

In [21]:
#The new suggested fluxes from the MOMA analysis are:

#Growth rate = 0.221
iBsu1147ogm.reactions.get_by_id("bio00006").upper_bound = 0.379
iBsu1147ogm.reactions.get_by_id("bio00006").lower_bound = 0.37
 
new_Fluxesog = {"E00096": (0, 1000), "E00068": (0, 1000), "E00117": (-6.844, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.01),
          "E00032": (-0.021, 1000), "E00009": (-2.099, 1000), "E00002": (-9.199, 1000), "E00004": (0, 6.028), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147ogm, new_Fluxesog)
iBsu1147ogm.optimize()

#cobra.io.write_sbml_model(iBsu1147ogm, "C:/Users/sofie/OneDrive/Documents/Master - Data/OGModelMan.sbml")

"""
#Running a FBA analysis with the MOMA-fluxes: 
cbmodelmannog2 = reframed.from_cobrapy(iBsu1147ogm)
solmannog2 = FBA(cbmodelmannog2)
#print(solcb.show_values())

SynthesisIDs = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006",  "E00096", "E00068", "E00017", "E00117", "E00046", "E00012", "E00008", "E00032", "E00009", "E00002", "E00004"]

for reactionID in SynthesisIDs:
    flux = solmannog2.values.get(reactionID)
    print(reactionID, round(flux,6))"""

'\n#Running a FBA analysis with the MOMA-fluxes: \ncbmodelmannog2 = reframed.from_cobrapy(iBsu1147ogm)\nsolmannog2 = FBA(cbmodelmannog2)\n#print(solcb.show_values())\n\nSynthesisIDs = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006",  "E00096", "E00068", "E00017", "E00117", "E00046", "E00012", "E00008", "E00032", "E00009", "E00002", "E00004"]\n\nfor reactionID in SynthesisIDs:\n    flux = solmannog2.values.get(reactionID)\n    print(reactionID, round(flux,6))'